<a href="https://colab.research.google.com/github/DaharaD/DSPL-PREPROCESSING/blob/main/Data_Preprocessing_DSPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
file_path = '/content/drive/MyDrive/Data preprocessing DSPL/hdx_hapi_food_price_lka (4).csv'
df = pd.read_csv(file_path)
df.head()

,location_code,has_hrp,in_gho,provider_admin1_name,provider_admin2_name,admin1_code,admin1_name,admin2_code,admin2_name,admin_level,...,lon,commodity_category,commodity_name,unit,price_flag,price_type,currency_code,price,reference_period_start,reference_period_end
0,#country+code,#meta+has_hrp,#meta+in_gho,#adm1+name+provider,#adm2+name+provider,#adm1+code,#adm1+name,#adm2+code,#adm2+name,#adm+level,...,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#date+start,#date+end
1,LKA,False,False,Western,Colombo,LK1,Western,LK11,Colombo,2,...,79.85,oil and fats,Oil (coconut),750 ML,actual,Retail,LKR,549.33,2023-03-15,2023-04-14
2,LKA,False,False,Western,Gampaha,LK1,Western,LK12,Gampaha,2,...,79.89,cereals and tubers,Potatoes (imported),KG,actual,Retail,LKR,159.35,2023-03-15,2023-04-14
3,LKA,False,False,Western,Gampaha,LK1,Western,LK12,Gampaha,2,...,79.89,cereals and tubers,Potatoes (local),KG,actual,Retail,LKR,339.2,2023-03-15,2023-04-14
4,LKA,False,False,Western,Gampaha,LK1,Western,LK12,Gampaha,2,...,79.89,"meat, fish and eggs",Eggs,Unit,actual,Retail,LKR,57.33,2023-03-15,2023-04-14


In [ ]:
# Remove duplicates
df = df.drop_duplicates()

In [ ]:
# Handle missing values
df = df.fillna(method='ffill').fillna(method='bfill')

<ipython-input-105-9e13153e90c8>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')


In [ ]:
# Rename columns to make them more understandable
df = df.rename(columns={
'provider_admin1_name': 'Provider_Admin1_Name',
'provider_admin2_name': 'Provider_Admin2_Name',
'admin1_name': 'Admin1_Name',
'admin2_name': 'Admin2_Name',
'market_name': 'Market_Name',
'lat': 'Latitude',
'lon': 'Longitude',
'commodity_category': 'Commodity_Category',
'commodity_name': 'Commodity_Name',
'unit': 'Unit',
'price_type': 'Price_Type',
'price': 'Price',
'reference_period_start': 'Reference_Period_Start',
'reference_period_end': 'Reference_Period_End'
})

In [ ]:
# Remove non-vital columns
columns_to_remove = ['location_code', 'has_hrp', 'in_gho', 'admin1_code', 'admin2_code', 'admin_level', 'price_flag', 'currency_code']
df = df.drop(columns=columns_to_remove)

In [ ]:
# Ensure the price column contains only numeric values
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# Drop rows with NaN values in the price column
df = df.dropna(subset=['Price'])

# Drop rows with NaT values in the date columns
df = df.dropna(subset=['Reference_Period_Start', 'Reference_Period_End'])


# Remove rows with null, NaN, or 'nun' values in any cell
df_cleaned = df.dropna()
df_cleaned = df_cleaned[~df_cleaned.apply(lambda row: row.astype(str).str.contains('nun')).any(axis=1)]

In [ ]:
# Standardize units
def convert_to_kg(row):
 if 'ML' in row['Unit']:
  return row['Price'] / 1000 # Convert ML to L (assuming density of water)
 elif 'KG' in row['Unit']:
  return row['Price'] # Already in KG
 elif 'Unit' in row['Unit']:
  return row['Price'] # Keep as is for unit items
 else:
  return row['Price'] # Default case

df['Standardized_Price'] = df.apply(convert_to_kg, axis=1)


In [ ]:
# Convert 'Reference_Period_Start' and 'Reference_Period_End' to datetime objects
df['Reference_Period_Start'] = pd.to_datetime(df['Reference_Period_Start'])
df['Reference_Period_End'] = pd.to_datetime(df['Reference_Period_End'])

# Extract the month from reference_period_start and reference_period_end
df['Start_Month'] = df['Reference_Period_Start'].dt.month
df['End_Month'] = df['Reference_Period_End'].dt.month

In [ ]:
df.head()

,Provider_Admin1_Name,Provider_Admin2_Name,Admin1_Name,Admin2_Name,Market_Name,Latitude,Longitude,Commodity_Category,Commodity_Name,Unit,Price_Type,Price,Reference_Period_Start,Reference_Period_End,Standardized_Price,Start_Month,End_Month
1,Western,Colombo,Western,Colombo,Colombo City,6.93,79.85,oil and fats,Oil (coconut),750 ML,Retail,549.33,2023-03-15,2023-04-14,0.54933,3,4
2,Western,Gampaha,Western,Gampaha,Economic Centre - Peliyagoda,6.97,79.89,cereals and tubers,Potatoes (imported),KG,Retail,159.35,2023-03-15,2023-04-14,159.35000,3,4
3,Western,Gampaha,Western,Gampaha,Economic Centre - Peliyagoda,6.97,79.89,cereals and tubers,Potatoes (local),KG,Retail,339.20,2023-03-15,2023-04-14,339.20000,3,4
4,Western,Gampaha,Western,Gampaha,Economic Centre - Peliyagoda,6.97,79.89,"meat, fish and eggs",Eggs,Unit,Retail,57.33,2023-03-15,2023-04-14,57.33000,3,4
5,Western,Gampaha,Western,Gampaha,Economic Centre - Peliyagoda,6.97,79.89,miscellaneous food,"Chili (red, dry raw)",KG,Retail,1404.84,2023-03-15,2023-04-14,1404.84000,3,4


In [ ]:
# Aggregate data by month
df_monthly = df.groupby(['Start_Month', 'Commodity_Name']).agg({'Standardized_Price': 'mean'}).reset_index()
df_monthly = df_monthly.rename(columns={'Standardized_Price': 'Monthly_Average_Price'})

In [ ]:
# Filter outliers (optional step)
df = df[(df['Standardized_Price'] < df['Standardized_Price'].mean() + 3 * df['Standardized_Price'].std()) & (df['Standardized_Price'] > df['Standardized_Price'].mean() - 3 * df['Standardized_Price'].std())]

In [ ]:
# Add descriptive statistics
df['Price_Mean'] = df.groupby('Commodity_Name')['Standardized_Price'].transform('mean')
df['Price_Median'] = df.groupby('Commodity_Name')['Standardized_Price'].transform('median')
df['Price_Std'] = df.groupby('Commodity_Name')['Standardized_Price'].transform('std')

In [ ]:

#Define the commodity mapping dictionary
commodity_mapping = {
 'Bananas': 'Vegetables and Fruits',
'Carrots': 'Vegetables and Fruits',
'Coconut': 'Vegetables and Fruits',
'Eggplants': 'Vegetables and Fruits',
'Onions (imported)': 'Vegetables and Fruits',
'Onions (red, local)': 'Vegetables and Fruits',
'Papaya': 'Vegetables and Fruits',
'Pineapples': 'Vegetables and Fruits',
'Pumpkin': 'Vegetables and Fruits',
'Snake gourd': 'Vegetables and Fruits',
'Tomatoes': 'Vegetables and Fruits',
'Potatoes (imported)': 'Cereals and Tubers',
'Potatoes (local)': 'Cereals and Tubers',
'Rice (medium grain)': 'Cereals and Tubers',
'Rice (white)': 'Cereals and Tubers',
'Eggs': 'Meat, Fish and Eggs',
'Fish (dry, sprats)': 'Meat, Fish and Eggs',
'Fish (goldstripe sardinella)': 'Meat, Fish and Eggs',
'Fish (sail fish)': 'Meat, Fish and Eggs',
'Fish (skipjack tuna)': 'Meat, Fish and Eggs',
'Fish (trenched sardinella)': 'Meat, Fish and Eggs',
'Fish (yellowfin tuna)': 'Meat, Fish and Eggs',
'Fish (jack)': 'Meat, Fish and Eggs',
'Meat (chicken, broiler)': 'Meat, Fish and Eggs',
'Meat (chicken, fresh)': 'Meat, Fish and Eggs',
'Beans': 'Pulses and Nuts',
'Beans (mung)': 'Pulses and Nuts',
'Cowpeas (whole, average)': 'Pulses and Nuts',
'Lentils': 'Pulses and Nuts',
'Oil (coconut)': 'Oil and Fats',
'Chili (red, dry raw)': 'Miscellaneous Food'
}

# Map the commodity names to broader categories
df['Commodity_Category'] = df['Commodity_Name'].map(commodity_mapping).fillna(df['Commodity_Category'])


In [ ]:
df.head()

,Provider_Admin1_Name,Provider_Admin2_Name,Admin1_Name,Admin2_Name,Market_Name,Latitude,Longitude,Commodity_Category,Commodity_Name,Unit,Price_Type,Price,Reference_Period_Start,Reference_Period_End,Standardized_Price,Start_Month,End_Month,Price_Mean,Price_Median,Price_Std
1,Western,Colombo,Western,Colombo,Colombo City,6.93,79.85,Oil and Fats,Oil (coconut),750 ML,Retail,549.33,2023-03-15,2023-04-14,0.54933,3,4,0.542627,0.5466,0.009346
2,Western,Gampaha,Western,Gampaha,Economic Centre - Peliyagoda,6.97,79.89,Cereals and Tubers,Potatoes (imported),KG,Retail,159.35,2023-03-15,2023-04-14,159.35000,3,4,219.640939,216.0000,26.978334
3,Western,Gampaha,Western,Gampaha,Economic Centre - Peliyagoda,6.97,79.89,Cereals and Tubers,Potatoes (local),KG,Retail,339.20,2023-03-15,2023-04-14,339.20000,3,4,401.322690,396.7350,40.321356
4,Western,Gampaha,Western,Gampaha,Economic Centre - Peliyagoda,6.97,79.89,"Meat, Fish and Eggs",Eggs,Unit,Retail,57.33,2023-03-15,2023-04-14,57.33000,3,4,45.809904,46.9500,7.860537
5,Western,Gampaha,Western,Gampaha,Economic Centre - Peliyagoda,6.97,79.89,Miscellaneous Food,"Chili (red, dry raw)",KG,Retail,1404.84,2023-03-15,2023-04-14,1404.84000,3,4,997.053691,921.4350,191.682532


In [ ]:
# Assuming df is your cleaned DataFrame
# Save the cleaned dataset to the specified file path
cleaned_file_path = '/content/drive/MyDrive/Data preprocessing DSPL/cleaned_hdx_hapi_food_price_lka.xlsx'  # Define the file path here
df.to_excel(cleaned_file_path, index=False)

print(f"Cleaned dataset saved to {cleaned_file_path}")

Cleaned dataset saved to /content/drive/MyDrive/Data preprocessing DSPL/cleaned_hdx_hapi_food_price_lka.xlsx
